In [1]:
# Creates a simple random distance matrix
randDistInput <- function(nCities, minDist = 1, maxDist = 100) {
    nRows <- nCities**2;
    randDistances <- sample(minDist:maxDist, nRows)
    distances <- matrix(randDistances, nCities, nCities)
    for(i in 1:nCities) {
        for(j in 1:nCities) {
            if(runif(1, min = 0, max = 1) < 1/(2*nCities)) {
                distances[i, j] <- Inf
            }
            distances[j, i] <- distances [i, j]
        }
        distances[i, i] <- 0
    }
    return(distances)
}

In [2]:
# Calcualtes all the possible permutations
permutations <- function(n) {
    if(n == 1) return(matrix(1))
    else {
        sp <- permutations(n - 1)
        p <- nrow(sp)
        A <- matrix(nrow = n*p, ncol = n)
        for(i in 1:n) {
            A[(i - 1)*p + 1:p, ] <- cbind(i, sp + (sp >= i))
        }
        return(A)
    }
}

In [3]:
getTotalDistance <- function(path, distances) {
    path <- c(path, path[1])
    totalDistance <- 0
    for(n in 1:(length(path) - 1)) {
        totalDistance <- totalDistance + distances[path[n], path[n + 1]]
    }
    return(totalDistance)
}

In [4]:
bruteForcePath <- function(distances) {
    shortestPathLength <- Inf
    nCities <- nrow(distances)
    possiblePaths <- cbind(c(1), permutations(nCities - 1) + 1)
    for(nPath in 1:nrow(possiblePaths)) {
        path <- possiblePaths[nPath, ]
        thisPathLength <- getTotalDistance(path, distances)
        if(thisPathLength < shortestPathLength) {
            minPathNumber <- nPath
            shortestPathLength <- thisPathLength
        } 
    }
    minPath <- possiblePaths[minPathNumber, ]
    return(minPath)
}

In [5]:
greedySolution <- function(distances) {
    nCities <- nrow(distances)
    visitedCities <- c(1)
    # choose next city
    while(length(visitedCities) < nCities) {
        currentCity <- tail(visitedCities, n = 1)
        neighboursDistances <- distances[currentCity, ]
        minDist <- Inf
        availableNeighbours <- 1:nCities
        availableNeighbours <- availableNeighbours[!(availableNeighbours %in% visitedCities)]
        for(neighbour in availableNeighbours) {
            thisDistance <- neighboursDistances[neighbour]
            if(thisDistance < minDist) {
                nextCity <- neighbour
                minDist <- thisDistance
            }
        }
        visitedCities <- c(visitedCities, nextCity)
    }
    return(visitedCities)
}

In [6]:
solveTSP <- function(distances, method = "auto") {
    cat("Distance matrix:\n")
    print(distances)
    
    startTime <- Sys.time()
    
    nCities = nrow(distances)
    nInf <- sum(is.infinite(distances))
    nRoads <- nCities**2 - nCities
    completeness <- 1 - nInf/nRoads
    
    if(method == "auto") {
        if(nCities < 9) {
            method <- "exact"
        }
        else {
            method <- "greedy"
        }
    }
    
    if(method == "exact") {
        solutionPath <- bruteForcePath(distances)
    }
    else if(method == "greedy") {
        solutionPath <- greedySolution(distances)
    }
    executionTime <- Sys.time() - startTime
    
    solutionDistance <- getTotalDistance(solutionPath, distances)
    
    cat("Completeness:", completeness,"\nMethod:", method, "\nCost:", solutionDistance, "\nExecution time:", executionTime, "\nPath:", solutionPath)
}

In [7]:
# Example input
exampleDistances <- as.matrix(read.table("./inputs/example1.csv", sep = ',', header = TRUE))
solveTSP(exampleDistances, method = "exact")
cat("\n\n")
solveTSP(exampleDistances, method = "greedy")

Distance matrix:
      V1 V2  V3 V4
[1,]   0 38 Inf 50
[2,]  38  0  88 71
[3,] Inf 88   0 29
[4,]  50 71  29  0
Completeness: 0.8333333 
Method: exact 
Cost: 205 
Execution time: 0.08492589 
Path: 1 2 3 4

Distance matrix:
      V1 V2  V3 V4
[1,]   0 38 Inf 50
[2,]  38  0  88 71
[3,] Inf 88   0 29
[4,]  50 71  29  0
Completeness: 0.8333333 
Method: greedy 
Cost: Inf 
Execution time: 0.03125691 
Path: 1 2 4 3

In [9]:
# Random input
randomDistances <- as.matrix(read.table("./inputs/random1.csv", sep = ',', header = TRUE))
solveTSP(randomDistances, method = "exact")
cat("\n\n")
solveTSP(randomDistances, method = "greedy")

Distance matrix:
     V1 V2 V3 V4  V5 V6  V7 V8
[1,]  0 68 48 45  90 30  35 51
[2,] 68  0 64 14  94 28  25 34
[3,] 48 64  0 36  17 96  19  2
[4,] 45 14 36  0  46 79  58 98
[5,] 90 94 17 46   0 74 Inf 56
[6,] 30 28 96 79  74  0   5 78
[7,] 35 25 19 58 Inf  5   0 18
[8,] 51 34  2 98  56 78  18  0
Completeness: 0.9642857 
Method: exact 
Cost: 190 
Execution time: 0.4111779 
Path: 1 6 2 4 5 3 8 7

Distance matrix:
     V1 V2 V3 V4  V5 V6  V7 V8
[1,]  0 68 48 45  90 30  35 51
[2,] 68  0 64 14  94 28  25 34
[3,] 48 64  0 36  17 96  19  2
[4,] 45 14 36  0  46 79  58 98
[5,] 90 94 17 46   0 74 Inf 56
[6,] 30 28 96 79  74  0   5 78
[7,] 35 25 19 58 Inf  5   0 18
[8,] 51 34  2 98  56 78  18  0
Completeness: 0.9642857 
Method: greedy 
Cost: 200 
Execution time: 0.001999855 
Path: 1 6 7 8 3 5 4 2

In [ ]:
# To do: implement 2-opt greedy algorithm